# Úkol č. 1 - vizualizace dat a web scraping (do 11. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.psp.cz/sqw/hlasovani.sqw?o=8 týkající hlasování současné poslanecké sněmovny, uložit tato data v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Data

 * Měli byste stáhnout data ze všech hlasování současné poslanecké sněmovny (tj. od voleb v roce 2017) a to až na úroveň jednotlivých poslanců. 
 * Data by měla obsahovat i stručný popis toho, o jaké hlasování šlo.

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data a uložte je do vhodného formátu, který se Vám bude dále dobře zpracovávat. 
  * **I když nemusíte mít sněmovnu rádi, snažte se nezahltit server a mezi požadavky na stránku mějte alespoň vteřinu mezeru.**
  * V druhé části Vašeho Jupyter notebooku pracujte se staženými daty v souborech, které jsou výsledkem Vašeho stahovacího skriptu (aby opravující nemusel spouštět stahování z webu).
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Odchody a přeběhnutí poslanců mezi jednotlivými stranami.
    * Jakou mají jednotliví poslanci docházku (účast na hlasování) a jak jsou na tom s docházkou strany jako celek.
    * Jak často spolu jednotlivé strany hlasují ve shodě a jak často hlasují odlišně.
    * Jak jsou poslanci jednotlivých stran jednotní v hlasování (jak často hlasují stejně, kdo jsou největší rebelové).

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * Ve vizualizacích a grafech nějak zachyťte časový vývoj (např. jak se mění docházka, shoda mezi stranami atp.).
  * Najděte jednotlivé poslance, kteří se nejvíce shodují při svém hlasování či kteří mají co nejpodobnější účast na hlasování.
  * Zkuste rozlišit v datech důležitá hlasování a zpracujte vizualizace (také) pro ně.
  * Zkuste najít hlasování, kde poslanci hlasovali nejvíce jinak, než obvykle.
  
## Tipy a triky
  * Balíčky importujte na začátku notebooku (příp. na začátku scrapovací a pak na začátku vizualizační části).
  * Využívejte markdown buňky (jako je ta, ve které je tento text) a naspisy, abyste usnadnily orientaci opravující(mu).
  * Pečlivě si vybírejte grafy a vizualizace, aby co nejlépe vyjádřily to, co vyjádřit chcete. Doporučuji projít si pro inspiraci alespoň galerie balíčků `matplotlib` a `seaborn`.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte Jupyter Notebook (příp. doplněn skripty), ale i soubor(y) se staženými daty (ať opravující nemusí stahovat data).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [109]:
import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

Při procházení stránek jednotlivých hlasování jsem si všiml, že v URL každého z nich je index, který určuje pořadí daného hlasování. Přechod mezi jedním hlasováním a druhým je vždy jen inkrement tohoto indexu, takže jsem si poznamenal indexy prvního a posledního hlasování, což mi následně pomůže doménu procházet efektivněji.

In [112]:
# important information extracted from website analysis

voting_start = 69445 # index hidden in the URL of the first voting
voting_end = 73901 # index in the URL of the last voting
char_set = 'windows-1250' # document character set for the website

# constants

VOTINGS_FILE = 'data/votings.csv'
INDIVIDUALS_FILE = 'data/individuals.csv'
PARTIES_FILE = 'data/parties.csv'
COLLECT_DATA = False
time = 3852.08399105072

In [111]:
def setup_files():
    f = open(VOTINGS_FILE, 'w', encoding='utf-8')
    f.write('Voting ID; Meeting number; Voting number; Subject; Number of voters; Yes; No; Unregistered; Refrained; Excused; Passed; Valid\n')
    f.close()
    
    f2 = open(INDIVIDUALS_FILE, 'w', encoding='utf-8')
    f2.write('Name; Party; Voting ID; Vote\n')
    f2.close()
    
    f3 = open(PARTIES_FILE, 'w', encoding='utf-8')
    f3.write('Party; Voting ID; Yes; No; Unregistered; Refrained; Excused\n')
    f3.close()


# function takes header of the website and returns a list including its basic information
# [meeting number, voting number, date, time, subject]
def split_header(header):
    match = re.search(r'[0-9][0-9]:[0-9][0-9]', header)
    if not match:
        raise Exception('Unable to split header')
    split = match.end()
    arr = [header[:split], header[split:]]
    res = [x.strip() for x in arr[0].split(',')]
    res.append(arr[1].strip() if len(arr[1]) > 0 else "''")
    res[0] = res[0].split('.')[0]
    res[1] = res[1].split('.')[0]
    return res
    

def process_voting(voting_index, soup):
    main = soup.find('div', { 'id' : 'main-content' })

    # process header
    voting_header = soup.find('h1', { 'class' : 'page-title-x' }).get_text()
    header = split_header(voting_header)
    f = open(VOTINGS_FILE, 'a', encoding='utf-8')
    f.write(f'{voting_index}; {header[0]}; {header[1]}; {header[4]}; ')

    # process vote summary table
    cell = soup.find_all('i', text=re.compile('Celkem'))[0].parent.parent
    curr = cell.next_sibling
    for i in range(6):
        f.write(f'{curr.get_text()}; ')
        curr = curr.next_sibling

    # vote passed?
    summary = soup.find('div', {'class' : 'summary'})
    result = summary.find('span')
    f.write('1; ') if result.get_text() == 'NÁVRH BYL PŘIJAT' else f.write('0; ')

    # valid election?
    status = soup.find_all('p', {'class': 'status invalid'})
    f.write('{valid}'.format(valid='0' if len(status) > 0 and status[0] == "Hlasování bylo prohlášeno za zmatečné." else '1\n'))
    f.close()
        

def process_party(voting_index, soup, party_info, text, name):
    f = open(PARTIES_FILE, 'a', encoding='utf-8')
    nums = party_info.find('span', {'class' : 'results'}).find_all('strong')
    
    i = 0
    if 'Ano:' in text:
        yes = nums[i].get_text()
        i += 1
    else:
        yes = 0
            
    if 'Ne:' in text:
        no = nums[i].get_text()
        i += 1
    else:
        no = 0
        
    if 'Nepřihlášen:' in text:
        unregistered = nums[i].get_text()
        i += 1
    else:
        unregistered = 0
        
    if 'Zdržel se:' in text:
        refrained = nums[i].get_text()
        i += 1
    else:
        refrained = 0
        
    if 'Omluven:' in text:
        excused = nums[i].get_text()
        i += 1
    else:
        excused = 0

    #print(f'{name}: {yes},{no},{unregistered},{refrained},{excused}')
    f.write(f'{name}; {voting_index}; {yes}; {no}; {unregistered}; {refrained}; {excused}\n')
    f.close()
        

# vote: A (yes), N (no), 0 (Unregistered), Z (refrained), M (excused)
def process_individuals(voting_index, soup):
    party_tables = soup.find('div', {'id' : 'main-content'}).find_all('h2', {'class' : 'section-title center'})[1:]
    res_tables = soup.find_all('ul', {'class' : 'results'})
    
    for i in range(len(party_tables) - 1):
        party_info = party_tables[i].find('span')
        text = party_info.get_text()
        if 'Tabulkovy vypis' in text:
            continue

        tmp = text.split(" ", 1)
        party_name = text.split(' ', 1)[0]
        process_party(voting_index, soup, party_info, text, party_name)
        
        party_voters = res_tables[i].find_all('li')
        f2 = open(INDIVIDUALS_FILE, 'a', encoding='utf-8')
        for voter in party_voters:
            name = voter.find('a').get_text()
            vote = voter.find('span').contents[0]
            #print(f'{name}, {party_name}, {vote}')
            f2.write(f'{name}; {party_name}; {voting_index}; {vote}\n')
        f2.close()

        
def process_page(voting_index):
    try:
        url = f'https://www.psp.cz/sqw/hlasy.sqw?g={voting_index}&l=cz'
        res = requests.get(url)
        html_page = res.content
        soup = BeautifulSoup(html_page, 'html.parser', from_encoding=char_set)
        
        process_voting(voting_index, soup)
        process_individuals(voting_index, soup)
    except Exception as e:
        print(f'Could not parse page: {voting_index} ... {e}')


def process_data():
    start = time.time()
    setup_files()
    
    for i in range(voting_start, voting_end):
        if i % 100 == 0:
            print(f'Processing page: {i}')
        process_page(i)
        time.sleep(0.3)
        
    end = time.time()
    print(end - start)

if COLLECT_DATA:
    process_data()

Processing page: 69500
Processing page: 69600
Processing page: 69700
Processing page: 69800
Processing page: 69900
Processing page: 70000
Processing page: 70100
Processing page: 70200
Processing page: 70300
Processing page: 70400
Processing page: 70500
Processing page: 70600
Processing page: 70700
Processing page: 70800
Processing page: 70900
Processing page: 71000
Processing page: 71100
Processing page: 71200
Processing page: 71300
Processing page: 71400
Processing page: 71500
Processing page: 71600
Processing page: 71700
Processing page: 71800
Processing page: 71900
Processing page: 72000
Processing page: 72100
Processing page: 72200
Processing page: 72300
Processing page: 72400
Processing page: 72500
Processing page: 72600
Processing page: 72700
Processing page: 72800
Processing page: 72900
Processing page: 73000
Processing page: 73100
Processing page: 73200
Processing page: 73300
Processing page: 73400
Processing page: 73500
Processing page: 73600
Processing page: 73700
Processing 

In [179]:
df_v = pd.read_csv('data/votings.csv', delimiter=';')
df_i = pd.read_csv('data/individuals.csv', delimiter=';')
df_p = pd.read_csv('data/parties.csv', delimiter=';')

df_p.loc[df_p['Voting ID'] == 70000]

,Party,Voting ID,Yes,No,Unregistered,Refrained,Excused
5546,ANO,70000,0,65,0,5,8
5547,ODS,70000,21,0,0,1,1
5548,Piráti,70000,21,0,0,1,0
5549,SPD,70000,17,0,0,0,2
5550,ČSSD,70000,0,10,0,3,2
5551,KSČM,70000,0,12,0,1,2
5552,KDU-ČSL,70000,9,0,0,0,1
5553,TOP09,70000,5,0,0,1,1
5554,STAN,70000,4,0,0,0,2
5555,Nezařaz,70000,4,0,0,0,1
